In [1]:
import pickle
import cv2
import hashlib
import time

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
start_time = time.time()

In [3]:
def frame_hash(frame):
    
    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate mean
    mean_val = gray_frame.mean()

    # Create a hash
    hash_val = hashlib.md5(str(mean_val).encode()).hexdigest()

    return hash_val

In [4]:
video_filename = f'../Queries/video11_1.mp4'
cap = cv2.VideoCapture(video_filename)

if not cap.isOpened():
    print(f"Could not open {video_filename}")
else:
    frame_hashes_all = []
    frame_hashes_10 = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Break the loop if the video ends
    
        # Process frame to get hash
        hash_val = frame_hash(frame)
    
        # Store hash value
        frame_hashes_all.append(hash_val)
        if frame_count % 10 == 0:
            frame_hashes_10.append(hash_val)

        frame_count += 1
        
    cap.release()
    cv2.destroyAllWindows()

In [5]:
# def is_video_contained(main_video_hashes, sub_video_hashes, similarity_threshold=0.95):
def is_video_contained(main_video_hashes, sub_video_hashes):
    len_sub = len(sub_video_hashes)
    len_main = len(main_video_hashes)

    for i in range(len_main - len_sub + 1):
        # matching_frames = 0
        # for j in range(len_sub):
        #     if main_video_hashes[i + j] == sub_video_hashes[j]:
        #         matching_frames += 1

        # similarity = matching_frames / len_sub
        # if similarity >= similarity_threshold:
        #     return True, i  # Returns True and the starting index in the main video

        if main_video_hashes[i:i+len_sub] == sub_video_hashes:
            return i

    return -1

In [6]:
vno = 0
for i in range(1, 21):
    fno = -1
    for j in range(10):
        filename = f'vpkl/v_{i}_{j}.pkl'
        with open(filename, 'rb') as file:
            v_10 = pickle.load(file)
            
        fno = is_video_contained(v_10, frame_hashes_10)
        
        if fno != -1:
            fno = j + 10 * fno
            break

    if fno != -1:
        filename = f'vpkl/v_{i}.pkl'
        with open(filename, 'rb') as file:
            v_all = pickle.load(file)

        if v_all[fno:fno+len(frame_hashes_all)] == frame_hashes_all:
            vno = i
            print('Video:', vno)
            print('Frame #:', fno - 1)
            break

Video: 11
Frame #: 8130


In [7]:
print('Time elapsed:', time.time() - start_time)

Time elapsed: 0.2219700813293457
